

#1.   Kết nối drive



In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive




#2.   Import thư viện



In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import joblib
from tensorflow.keras.models import load_model





#3. Đọc flie



In [3]:

file_path = "/content/drive/MyDrive/CNPM/Final_Augmented_dataset_Diseases_and_Symptoms.csv"  # Đường dẫn file trong Drive
df = pd.read_csv(file_path)



#4. Tiền xử lý dữ liệu




In [4]:
# Hiển thị toàn bộ cột
pd.set_option('display.max_columns', None)

# Hiển thị toàn bộ dòng
pd.set_option('display.max_rows', None)

# Không giới hạn độ rộng của dòng in ra
pd.set_option('display.max_colwidth', None)

# Không tóm tắt dataframe khi in ra
pd.set_option('display.expand_frame_repr', False)

###4.1 Kiểm tra số lượng bệnh và số dòng dữ liệu cho mỗi bệnh đó


In [5]:
# Kiểm tra danh sách bệnh duy nhất
unique_diseases = df["diseases"].unique()
print("Danh sách bệnh:", unique_diseases)

# Đếm số lượng bệnh duy nhất
num_diseases = df["diseases"].nunique()
print(f"Số lượng bệnh trong dataset: {num_diseases}")

# Đếm số dòng cho mỗi bệnh bằng value_counts()
disease_counts = df["diseases"].value_counts()

# In kết quả
print(disease_counts)


Danh sách bệnh: ['panic disorder' 'vocal cord polyp' 'turner syndrome' 'cryptorchidism'
 'poisoning due to ethylene glycol' 'atrophic vaginitis'
 'fracture of the hand' 'cellulitis or abscess of mouth'
 'eye alignment disorder' 'headache after lumbar puncture'
 'pyloric stenosis' 'salivary gland disorder' 'osteochondrosis'
 'injury to the knee' 'metabolic disorder' 'vaginitis'
 'sick sinus syndrome' 'tinnitus of unknown cause' 'glaucoma'
 'eating disorder' 'transient ischemic attack' 'pyelonephritis'
 'rotator cuff injury' 'chronic pain disorder' 'problem during pregnancy'
 'liver cancer' 'atelectasis' 'injury to the hand' 'choledocholithiasis'
 'injury to the hip' 'cirrhosis' 'thoracic aortic aneurysm'
 'subdural hemorrhage' 'diabetic retinopathy' 'fibromyalgia'
 'ischemia of the bowel' 'fetal alcohol syndrome' 'peritonitis'
 'injury to the abdomen' 'acute pancreatitis' 'thrombophlebitis' 'asthma'
 'foreign body in the vagina' 'restless leg syndrome' 'emphysema'
 'cysticercosis' 'indu

In [6]:
# Check shape
print(df.shape)  # (số dòng, 378) → 377 triệu chứng + 1 cột bệnh

# Check missing values
print(df.isnull().sum().sort_values(ascending=False))

(246945, 378)
neck weakness                                     0
diseases                                          0
anxiety and nervousness                           0
depression                                        0
shortness of breath                               0
depressive or psychotic symptoms                  0
sharp chest pain                                  0
dizziness                                         0
insomnia                                          0
abnormal involuntary movements                    0
hand or finger cramps or spasms                   0
foot or toe weakness                              0
wrinkles on skin                                  0
redness in or around nose                         0
tongue pain                                       0
joint swelling                                    0
throat redness                                    0
muscle weakness                                   0
elbow lump or mass                                

In [7]:
# Xem 5 dòng đầu của dts
df.head()

,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,palpitations,irregular heartbeat,breathing fast,hoarse voice,sore throat,difficulty speaking,cough,nasal congestion,throat swelling,diminished hearing,lump in throat,throat feels tight,difficulty in swallowing,skin swelling,retention of urine,groin mass,leg pain,hip pain,suprapubic pain,blood in stool,lack of growth,emotional symptoms,elbow weakness,back weakness,pus in sputum,symptoms of the scrotum and testes,swelling of scrotum,pain in testicles,flatulence,pus draining from ear,jaundice,mass in scrotum,white discharge from eye,irritable infant,abusing alcohol,fainting,hostile behavior,drug abuse,sharp abdominal pain,feeling ill,vomiting,headache,nausea,diarrhea,vaginal itching,vaginal dryness,painful urination,involuntary urination,pain during intercourse,frequent urination,lower abdominal pain,vaginal discharge,blood in urine,hot flashes,intermenstrual bleeding,hand or finger pain,wrist pain,hand or finger swelling,arm pain,wrist swelling,arm stiffness or tightness,arm swelling,hand or finger stiffness or tightness,wrist stiffness or tightness,lip swelling,toothache,abnormal appearing skin,skin lesion,acne or pimples,dry lips,facial pain,mouth ulcer,skin growth,eye deviation,diminished vision,double vision,cross-eyed,symptoms of eye,pain in eye,eye moves abnormally,abnormal movement of eyelid,foreign body sensation in eye,irregular appearing scalp,swollen lymph nodes,back pain,neck pain,low back pain,pain of the anus,pain during pregnancy,pelvic pain,impotence,infant spitting up,vomiting blood,regurgitation,burning abdominal pain,restlessness,symptoms of infants,wheezing,peripheral edema,neck mass,ear pain,jaw swelling,mouth dryness,neck swelling,knee pain,foot or toe pain,bowlegged or knock-kneed,ankle pain,bones are painful,knee weakness,elbow pain,knee swelling,skin moles,knee lump or mass,weight gain,problems with movement,knee stiffness or tightness,leg swelling,foot or toe swelling,heartburn,smoking problems,muscle pain,infant feeding problem,recent weight loss,problems with shape or size of breast,underweight,difficulty eating,scanty menstrual flow,vaginal pain,vaginal redness,vulvar irritation,weakness,decreased heart rate,increased heart rate,bleeding or discharge from nipple,ringing in ear,plugged feeling in ear,itchy ear(s),frontal headache,fluid in ear,neck stiffness or tightness,spots or clouds in vision,eye redness,lacrimation,itchiness of eye,blindness,eye burns or stings,itchy eyelid,feeling cold,decreased appetite,excessive appetite,excessive anger,loss of sensation,focal weakness,slurring words,symptoms of the face,disturbance of memory,paresthesia,side pain,fever,shoulder pain,shoulder stiffness or tightness,shoulder weakness,arm cramps or spasms,shoulder swelling,tongue lesions,leg cramps or spasms,abnormal appearing tongue,ache all over,lower body pain,problems during pregnancy,spotting or bleeding during pregnancy,cramps and spasms,upper abdominal pain,stomach bloating,changes in stool appearance,unusual color or odor to urine,kidney mass,swollen abdomen,symptoms of prostate,leg stiffness or tightness,difficulty breathing,rib pain,joint pain,muscle stiffness or tightness,pallor,hand or finger lump or mass,chills,groin pain,fatigue,abdominal distention,regurgitation.1,symptoms of the kidneys,melena,flushing,coughing up sputum,seizures,delusions or hallucinations,shoulder cramps or spasms,joint stiffness or tightness,pain or soreness of breast,excessive urination at night,bleeding from eye,rectal bleeding,constipation,temper problems,coryza,wrist weakness,eye strain,hemoptysis,lymphedema,skin on leg or foot looks infected,allergic reaction,congestion in chest,muscle swelling,pus in urine,abnormal size or shape of ear,low back weakness,sleepiness,apnea,abnormal breathing sounds,excessive growth,elbow cramps or spasms,feeling h

###4.2 Xóa các bệnh có ít hơn 100 mẫu

In [8]:
min_count =500
valid_diseases = disease_counts[disease_counts >= min_count].index

# Giữ lại các dòng có bệnh trong danh sách đủ điều kiện
df_filtered = df[df['diseases'].isin(valid_diseases)]
print(f"Số dòng trước khi lọc: {len(df)}")
print(f"Số dòng sau khi lọc: {len(df_filtered)}")
print(f"Số bệnh còn lại: {df_filtered['diseases'].nunique()}")


Số dòng trước khi lọc: 246945
Số dòng sau khi lọc: 168499
Số bệnh còn lại: 201


### 4.3 Xóa các symptons (triệu chứng) chỉ có giá trị 0 trong cột


In [9]:
# Kiểm tra các triệu chứng có giá trị 0 trong tất cả các dòng sau khi lọc bệnh
cols_to_drop = [col for col in df_filtered.columns if df_filtered[col].sum() == 0]

# Loại bỏ các cột có giá trị 0 trong toàn bộ dữ liệu đã lọc
df_cleaned = df_filtered.drop(columns=cols_to_drop)

# In ra các cột đã bị loại bỏ
print("Các triệu chứng đã bị loại bỏ:", cols_to_drop)

# In kết quả sau khi làm sạch
print(f"Số dòng sau khi làm sạch: {len(df_cleaned)}")


Các triệu chứng đã bị loại bỏ: ['lump in throat', 'emotional symptoms', 'elbow weakness', 'back weakness', 'pus in sputum', 'vaginal dryness', 'wrist stiffness or tightness', 'dry lips', 'eye deviation', 'cross-eyed', 'eye moves abnormally', 'infant spitting up', 'symptoms of infants', 'bowlegged or knock-kneed', 'recent weight loss', 'problems with shape or size of breast', 'underweight', 'difficulty eating', 'scanty menstrual flow', 'neck stiffness or tightness', 'itchy eyelid', 'feeling cold', 'shoulder weakness', 'arm cramps or spasms', 'shoulder swelling', 'abnormal appearing tongue', 'swollen abdomen', 'muscle stiffness or tightness', 'pallor', 'abdominal distention', 'flushing', 'shoulder cramps or spasms', 'joint stiffness or tightness', 'wrist weakness', 'eye strain', 'muscle swelling', 'pus in urine', 'abnormal size or shape of ear', 'low back weakness', 'excessive growth', 'elbow cramps or spasms', 'feeling hot and cold', 'absence of menstruation', 'low back cramps or spasms

In [10]:
# Đếm số cột còn lại trong df_cleaned
num_columns_left = df_cleaned.shape[1]

print(f"Số cột còn lại: {num_columns_left}")


Số cột còn lại: 264


###4.4 Thêm cột khoa tương ứng với các bệnh

In [11]:
disease_to_specialty = {
    "panic disorder": "Psychiatry",
    "vaginitis": "Obstetrics and Gynecology",
    "transient ischemic attack": "Neurology",
    "pyelonephritis": "Nephrology",
    "chronic pain disorder": "General Medicine",
    "problem during pregnancy": "Obstetrics and Gynecology",
    "fibromyalgia": "Rheumatology",
    "acute pancreatitis": "Gastroenterology",
    "asthma": "Pulmonology",
    "infectious gastroenteritis": "Gastroenterology",
    "acute sinusitis": "Otorhinolaryngology (ENT)",
    "chalazion": "Ophthalmology",
    "vaginal yeast infection": "Obstetrics and Gynecology",
    "ingrown toe nail": "Orthopedics",
    "corneal disorder": "Ophthalmology",
    "stroke": "Neurology",
    "hyperkalemia": "Endocrinology",
    "cornea infection": "Ophthalmology",
    "chronic sinusitis": "Otorhinolaryngology (ENT)",
    "conductive hearing loss": "Otorhinolaryngology (ENT)",
    "marijuana abuse": "Psychiatry",
    "bursitis": "Rheumatology",
    "actinic keratosis": "Dermatology",
    "acute otitis media": "Otorhinolaryngology (ENT)",
    "chronic obstructive pulmonary disease (copd)": "Pulmonology",
    "spondylosis": "Rheumatology",
    "injury to the shoulder": "Orthopedics",
    "heart block": "Cardiology",
    "attention deficit hyperactivity disorder (adhd)": "Psychiatry",
    "injury to the arm": "Orthopedics",
    "complex regional pain syndrome": "Rheumatology",
    "injury to the trunk": "Orthopedics",
    "vulvodynia": "Obstetrics and Gynecology",
    "concussion": "Neurology",
    "neuralgia": "Neurology",
    "bipolar disorder": "Psychiatry",
    "hypoglycemia": "Endocrinology",
    "hiatal hernia": "Gastroenterology",
    "allergy": "Allergy",
    "chronic otitis media": "Otorhinolaryngology (ENT)",
    "lymphedema": "General Medicine",
    "acute bronchospasm": "Pulmonology",
    "sciatica": "Rheumatology",
    "degenerative disc disease": "Rheumatology",
    "diverticulosis": "Gastroenterology",
    "pain after an operation": "Orthopedics",
    "fungal infection of the skin": "Dermatology",
    "injury to the leg": "Orthopedics",
    "mitral valve disease": "Cardiology",
    "parkinson disease": "Neurology",
    "gout": "Rheumatology",
    "otitis media": "Otorhinolaryngology (ENT)",
    "drug abuse (opioids)": "Psychiatry",
    "acute kidney injury": "Urology",
    "threatened pregnancy": "Obstetrics and Gynecology",
    "gum disease": "Dentistry",
    "gastrointestinal hemorrhage": "Gastroenterology",
    "seborrheic dermatitis": "Dermatology",
    "labyrinthitis": "Otorhinolaryngology (ENT)",
    "anxiety": "Psychiatry",
    "bell palsy": "Neurology",
    "conjunctivitis due to allergy": "Ophthalmology",
    "drug reaction": "Allergy",
    "osteoarthritis": "Rheumatology",
    "macular degeneration": "Ophthalmology",
    "pneumonia": "Pulmonology",
    "fracture of the rib": "Orthopedics",
    "vaginal cyst": "Obstetrics and Gynecology",
    "alcohol abuse": "Psychiatry",
    "carpal tunnel syndrome": "Rheumatology",
    "kidney stone": "Urology",
    "chronic glaucoma": "Ophthalmology",
    "nose disorder": "Otorhinolaryngology (ENT)",
    "dental caries": "Dentistry",
    "muscle spasm": "Rheumatology",
    "itching of unknown cause": "Dermatology",
    "corneal abrasion": "Ophthalmology",
    "hypertensive heart disease": "Cardiology",
    "alcohol withdrawal": "Psychiatry",
    "hemiplegia": "Neurology",
    "seasonal allergies (hay fever)": "Allergy",
    "dry eye of unknown cause": "Ophthalmology",
    "fungal infection of the hair": "Dermatology",
    "tooth abscess": "Dentistry",
    "rectal disorder": "Gastroenterology",
    "stye": "Ophthalmology",
    "oral mucosal lesion": "Dentistry",
    "intracerebral hemorrhage": "Neurology",
    "ganglion cyst": "Orthopedics",
    "skin disorder": "Dermatology",
    "tooth disorder": "Dentistry",
    "heart attack": "Cardiology",
    "fracture of the leg": "Orthopedics",
    "oral thrush (yeast infection)": "Dentistry",
    "obstructive sleep apnea (osa)": "Pulmonology",
    "psoriasis": "Dermatology",
    "arthritis of the hip": "Rheumatology",
    "sensorineural hearing loss": "Otorhinolaryngology (ENT)",
    "sickle cell crisis": "Hematology"
}
# Cập nhật dictionary bệnh và khoa tương ứng
disease_to_specialty.update({
    "prostatitis": "Urology",
    "otitis externa (swimmer's ear)": "Otorhinolaryngology (ENT)",
    "acute bronchiolitis": "Pulmonology",
    "insect bite": "Dermatology",
    "paroxysmal ventricular tachycardia": "Cardiology",
    "pyogenic skin infection": "Dermatology",
    "viral exanthem": "Dermatology",
    "noninfectious gastroenteritis": "Gastroenterology",
    "benign prostatic hyperplasia (bph)": "Urology",
    "cataract": "Ophthalmology",
    "spinal stenosis": "Rheumatology",
    "acute bronchitis": "Pulmonology",
    "ischemic heart disease": "Cardiology",
    "croup": "Pulmonology",
    "idiopathic excessive menstruation": "Obstetrics and Gynecology",
    "ear drum damage": "Otorhinolaryngology (ENT)",
    "erectile dysfunction": "Urology",
    "temporary or benign blood in urine": "Urology",
    "onychomycosis": "Dermatology",
    "iron deficiency anemia": "Hematology",
    "tendinitis": "Rheumatology",
    "common cold": "Pulmonology",
    "depression": "Psychiatry",
    "drug abuse": "Psychiatry",
    "urinary tract obstruction": "Urology",
    "varicocele of the testicles": "Urology",
    "irritable bowel syndrome": "Gastroenterology",
    "lipoma": "Orthopedics",
    "rheumatoid arthritis": "Rheumatology",
    "impetigo": "Dermatology",
    "idiopathic irregular menstrual cycle": "Obstetrics and Gynecology",
    "interstitial lung disease": "Pulmonology",
    "mononeuritis": "Neurology",
    "schizophrenia": "Psychiatry",
    "sepsis": "General Medicine",
    "cholecystitis": "Gastroenterology",
    "social phobia": "Psychiatry",
    "ear wax impaction": "Otorhinolaryngology (ENT)",
    "rosacea": "Dermatology",
    "post-traumatic stress disorder (ptsd)": "Psychiatry",
    "blepharitis": "Ophthalmology",
    "cystitis": "Urology",
    "hemorrhoids": "Gastroenterology",
    "contact dermatitis": "Dermatology",
    "sinus bradycardia": "Cardiology",
    "pelvic inflammatory disease": "Obstetrics and Gynecology",
    "liver disease": "Hepatology",
    "chronic constipation": "Gastroenterology",
    "flu": "Pulmonology",
    "skin polyp": "Dermatology",
    "brachial neuritis": "Neurology",
    "esophagitis": "Gastroenterology",
    "diverticulitis": "Gastroenterology",
    "psychotic disorder": "Psychiatry",
    "smoking or tobacco addiction": "Psychiatry",
    "sprain or strain": "Orthopedics",
    "idiopathic painful menstruation": "Obstetrics and Gynecology",
    "eustachian tube dysfunction (ear disorder)": "Otorhinolaryngology (ENT)",
    "appendicitis": "Gastroenterology",
    "hyperemesis gravidarum": "Obstetrics and Gynecology",
    "pleural effusion": "Pulmonology",
    "sialoadenitis": "Otorhinolaryngology (ENT)",
    "urinary tract infection": "Urology",
    "ankylosing spondylitis": "Rheumatology",
    "peripheral nerve disorder": "Neurology",
    "sebaceous cyst": "Dermatology",
    "neurosis": "Psychiatry",
    "seborrheic keratosis": "Dermatology",
    "spontaneous abortion": "Obstetrics and Gynecology",
    "adjustment reaction": "Psychiatry",
    "gallstone": "Gastroenterology",
    "multiple sclerosis": "Neurology",
    "angina": "Cardiology",
    "skin pigmentation disorder": "Dermatology",
    "diabetic ketoacidosis": "Endocrinology",
    "personality disorder": "Psychiatry",
    "strep throat": "Otorhinolaryngology (ENT)",
    "gastritis": "Gastroenterology",
    "skin cancer": "Dermatology",
    "developmental disability": "Psychiatry",
    "acute stress reaction": "Psychiatry",
    "delirium": "Psychiatry",
    "callus": "Dermatology",
    "deviated nasal septum": "Otorhinolaryngology (ENT)",
    "hemangioma": "Dermatology",
    "chronic back pain": "Rheumatology",
    "heart failure": "Cardiology",
    "conjunctivitis": "Ophthalmology",
    "herniated disk": "Rheumatology",
    "breast infection (mastitis)": "Obstetrics and Gynecology",
    "pulmonary embolism": "Pulmonology",
    "conduct disorder": "Psychiatry",
    "epididymitis": "Urology",
    "molluscum contagiosum": "Dermatology",
    "benign vaginal discharge (leukorrhea)": "Obstetrics and Gynecology",
    "laryngitis": "Otorhinolaryngology (ENT)",
    "diaper rash": "Dermatology",
    "gastroduodenal ulcer": "Gastroenterology",
    "eczema": "Dermatology",
    "white blood cell disease": "Hematology",
    "spondylolisthesis": "Rheumatology",
    "conjunctivitis due to virus": "Ophthalmology"
})




In [12]:
# Ánh xạ từng bệnh trong cột 'diseases' sang chuyên khoa tương ứng bằng từ điển 'disease_to_specialty'
df_cleaned['specialty'] = df_cleaned['diseases'].map(disease_to_specialty)

In [13]:
# print(df_cleaned.head())

In [14]:
# Kiểm tra nếu có giá trị NaN trong cột 'specialty'
has_nan_in_specialty = df_cleaned['specialty'].isna().any()

if has_nan_in_specialty:
    print("Cột 'specialty' có chứa NaN.")
else:
    print("Cột 'specialty' không có NaN.")

Cột 'specialty' không có NaN.


### 4.5 One-hot encoding cho diseases và specialty

In [15]:
# Tạo encoder
disease_encoder = LabelEncoder()
specialty_encoder = LabelEncoder()

# Fit và transform
df_cleaned['diseases_encoded'] = disease_encoder.fit_transform(df_cleaned['diseases'])
df_cleaned['specialty_encoded'] = specialty_encoder.fit_transform(df_cleaned['specialty'])

# X là các đặc trưng, y là nhãn đã mã hóa (diseases_encoded)
X = df_cleaned.drop(columns=['diseases', 'specialty', 'specialty_encoded', 'diseases_encoded'])  # X không bao gồm bệnh và chuyên khoa
y = df_cleaned['diseases_encoded']  # y là nhãn bệnh đã mã hóa

# One-hot encode y (y là label bệnh đã mã hóa)
y_onehot = to_categorical(y)


#5. Huấn luyện mô hình


###5.1 Chia tập dữ liệu

In [16]:
# Tách tập train+val và test trước
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Tách tiếp train và val từ tập train+val
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val)
# One-hot encode y cho MLP
y_train_oh = to_categorical(y_train)
y_val_oh = to_categorical(y_val)
y_test_oh = to_categorical(y_test)


###5.2 Xuất các file để sử dụng trong viết fastapi



In [17]:
# Lấy danh sách tên các cột triệu chứng từ DataFrame X và lưu lại dưới dạng file pickle
# File symptom_columns.pkl chứa danh sách các triệu chứng (tên cột) dùng cho việc huấn luyện và dự đoán
symptom_columns = list(X.columns)
joblib.dump(symptom_columns, '/content/drive/MyDrive/Mining/symptom_columns.pkl')

# Lưu các bộ mã hóa (encoder) dùng để biến đổi nhãn bệnh (disease) và chuyên khoa (specialty)
# Các file disease_encoder.pkl và specialty_encoder.pkl chứa mô hình encoder để mã hóa/chuyển đổi nhãn sang số
joblib.dump(disease_encoder, '/content/drive/MyDrive/Mining/disease_encoder.pkl')

# Tạo DataFrame mới chỉ gồm hai cột 'diseases' và 'specialty', loại bỏ các bản ghi trùng lặp
# DataFrame này chứa ánh xạ duy nhất từ từng bệnh sang chuyên khoa tương ứng
df_disease_specialty = df_cleaned[['diseases', 'specialty']].drop_duplicates()

# Lưu DataFrame ánh xạ bệnh - chuyên khoa dưới dạng file pickle để tái sử dụng dễ dàng
# File disease_specialty.pkl sẽ được dùng khi cần tra cứu hoặc mapping từ bệnh sang chuyên khoa
df_disease_specialty.to_pickle('/content/drive/MyDrive/Mining/disease_specialty.pkl')

# Hiển thị 100 dòng đầu tiên của DataFrame để kiểm tra dữ liệu vừa tạo
df_disease_specialty.head(100)

,diseases,specialty
0,panic disorder,Psychiatry
2503,vaginitis,Obstetrics and Gynecology
4841,transient ischemic attack,Neurology
5496,pyelonephritis,Nephrology
6510,chronic pain disorder,General Medicine
7187,problem during pregnancy,Obstetrics and Gynecology
9925,fibromyalgia,Rheumatology
11316,acute pancreatitis,Gastroenterology
12974,asthma,Pulmonology
14974,infectious gastroenteritis,Gastroenterology


###5.3 Huấn luyện mô hình MLP

In [18]:

# Build MLP model
mlp_model = Sequential([
    Dense(256, input_shape=(X_train.shape[1],), activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_train_oh.shape[1], activation='softmax')
])

mlp_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train
mlp_model.fit(X_train, y_train_oh, epochs=50, batch_size=32, validation_data=(X_val, y_val_oh))

# Lưu model
mlp_model.save('/content/drive/MyDrive/Mining/mlp_model.keras')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
3370/3370 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.6036 - loss: 1.6539 - val_accuracy: 0.8576 - val_loss: 0.3553
Epoch 2/50
3370/3370 ━━━━━━━━━━━━━━━━━━━━ 23s 7ms/step - accuracy: 0.8150 - loss: 0.5476 - val_accuracy: 0.8572 - val_loss: 0.3441
Epoch 3/50
3370/3370 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8256 - loss: 0.4959 - val_accuracy: 0.8602 - val_loss: 0.3252
Epoch 4/50
3370/3370 ━━━━━━━━━━━━━━━━━━━━ 24s 7ms/step - accuracy: 0.8313 - loss: 0.4639 - val_accuracy: 0.8600 - val_loss: 0.3227
Epoch 5/50
3370/3370 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - accuracy: 0.8325 - loss: 0.4528 - val_accuracy: 0.8575 - val_loss: 0.3242
Epoch 6/50
3370/3370 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - accuracy: 0.8391 - loss: 0.4301 - val_accuracy: 0.8624 - val_loss: 0.3130
Epoch 7/50
3370/3370 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8420 - loss: 0.4123 - val_accuracy: 0.8633 - val_loss: 0.3090
Epoch 8/50
3370/3370 ━━━━━━━━━━━━━━━━━━━━ 24s 7ms/step - accuracy: 0.8416 - loss: 0

###5.4 Load và kiểm tra mô hình


In [21]:
# Load models
mlp_model = load_model('/content/drive/MyDrive/Mining/mlp_model.keras')

test_case_1 = ["fever", "cough", "shortness of breath"]
# => thường liên quan hô hấp/phổi

test_case_2 = ["headache", "nausea", "vomiting"]
# => thường gặp trong tiêu hóa/tiêu hóa thần kinh

test_case_3 = ["abnormal appearing skin", "skin rash", "itching of skin"]
# => chuyên khoa da liễu

test_case_4 = ["vaginal discharge", "painful urination", "lower abdominal pain"]
# => sản phụ khoa/niệu

test_case_5 = ["back pain", "neck stiffness or tightness", "shoulder pain"]
# => cơ xương khớp

test_case_6 = ["delusions or hallucinations", "anxiety and nervousness", "depression"]
# => tâm thần kinh

test_case_7 = ["diarrhea", "blood in stool", "flatulence"]
# => tiêu hóa

test_case_8 = ["eye redness", "itchiness of eye", "lacrimation"]
# => chuyên khoa mắt

test_case_9 = ["irregular heartbeat", "palpitations", "chest tightness"]
# => tim mạch

test_case_10 = ["leg pain", "leg swelling", "ankle pain"]
# => cơ xương khớp, ngoại khoa

# Thêm custom nếu cần:
custom_case = ["fever", "headache", "dizziness", "skin rash"]


In [26]:
symptoms_input = test_case_3  # đổi case ở đây

input_vector = [1 if symptom in symptoms_input else 0 for symptom in symptom_columns]
input_df = pd.DataFrame([input_vector], columns=symptom_columns)

# Predict
predictions_mlp = mlp_model.predict(input_df)

# Lấy chỉ số bệnh có xác suất cao nhất
predicted_class_mlp = np.argmax(predictions_mlp, axis=1)

# Giải mã tên bệnh
predicted_disease_mlp = disease_encoder.inverse_transform(predicted_class_mlp)
print("Dự đoán bệnh:", predicted_disease_mlp[0])

specialty_pred_mlp = df_cleaned[df_cleaned['diseases'] == predicted_disease_mlp[0]]['specialty'].iloc[0]
print("Chuyên khoa:", specialty_pred_mlp)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Dự đoán bệnh: psoriasis
Chuyên khoa: Dermatology
